# **Chapter 6: Precognition (Thinking Step by Step)**

---
**Lesson:**

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to think through your answer first. 
Guess what? Claude is the same way.
Giving Claude time to think step by step sometimes makes Claude more accurate, particularly for complex tasks.

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

**Example 6.1 - Movie Review**

In the movie review prompt below, it's clear to a human reader that the second sentence belies the first. But Claude takes the word "unrelated" too literally.

In [ ]:
response = textgen_llm("""

Human: Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since the year 1900.

Assistant:""")

print_ww(response)

**Example 6.2 - Better movie review**

To improve Claude's response, let's allow Claude to think things out first before answering. We do that by literally spelling out the steps that Claude should take in order to process and think through the prompt. 
Now, Claude's response is much better and more nuanced.

In [ ]:
response = textgen_llm("""

Human: Is this review sentiment positive or negative? First write the best arguments for each side in <positive-arg> and <negative-arg> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.

Assistant:""")

print_ww(response)

**Example 6.3 - Blockbusters**

Letting Claude think can shift Claude's answer from incorrect to correct. It's that simple in many cases where Claude makes mistakes!
Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.	

In [ ]:
response = textgen_llm("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country?

Assistant:""")

print_ww(response)

**Example 6.4 - Blockbusters 2**

Let's fix this by asking Claude to think step by step. Instead of spelling out each step for Claude, you can also simply use those exact words and ask Claude to think step by step first.			

In [ ]:
response = textgen_llm("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country? Make assumptions where needed, and think step by step.

Assistant:""")

print_ww(response)

**Example 6.5 - Simon says**

It's important to note that to some degree, "let's think step by step" has been pre-built into Claude's training. You will sometimes see Claude thinking step by step even when you don't ask it to, particularly if you asked it a logic or math problem. However, even then, such techniques are not failproof, and Claude thinks step by step and still gets the wrong answer. To help Claude here, we can tell Claude exactly what process it should undertake to think through the problem.

In [ ]:
response = textgen_llm("""

Human: Use the following clues to answer the following multiple-choice question, using the following procedure:
(1) First, go through the clues one by one and consider whether the clue is potentially relevant
(2) Second, combine the relevant clues pay attention to every single word. And then reason out the answer to the question
(3) Third, map the answer to one of the multiple choice answers: either (a), (b), or (c)
 
Clues:
1. Miss Scarlett was the only person in the lounge.
2. The person with the pipe was in the kitchen.
3. Colonel Mustard was the only person in the observatory.
4. Professor Plum was not in the library nor the billiard room.
5. The person with the candlestick was in the observatory.
 
Question: Was Colonel Mustard in the observatory with the candlestick?
(a) Yes; Colonel Mustard was in the observatory with the candlestick
(b) No; Colonel Mustard was not in the observatory with the candlestick
(c) Unknown; there is not enough information to determine whether Colonel Mustard was in the observatory with the candlestick
 
Assistant:""")

print_ww(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 6.1 - Classifying Emails**

In this exercise, we'll be instructing Claude to sort emails into the following categories:
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)

For the first part of the exercise, change the prompt in the cell below to make Claude output the correct classification. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.

> *Tip: Use precognition and other techniques you've learned leading up to this chapter!*

In [ ]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.") #Should be classified as (B)

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?") #Should be classified as (A)

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WHAT IS GOING ON???") #Should be classified as (C)

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="How did I get here I am not good with computer.  Halp.") #Should be classified as (D)

response = textgen_llm(prompt)

print_ww(response)

# Chapter 6 - END.